# Carga de Datos


## .xls / HTML a .csv
Los datos del Mineduc se exportan como archivos .xls, realmente teniendo contenido de HTML. Debido a eso, debemos de parsear el archivo HTML e identificar la tabla correcta que contiene los datos. Luego, podemos exportar los datos a su archivo .csv correspondiente, utilizando el valor de la columna 'Departamento' para nombrarlo. Adicionalmente, los archivos utilizan un encoding diferente al estándar utf-8, por lo cual vamos a especificarlo al momento de leer los archivos HTML.

In [41]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
import re


def parse_html_excel_file(file_path):
    file_path = Path(file_path)

    try:
        tables = pd.read_html(str(file_path), encoding="iso-8859-1")

        if not tables:
            return {"success": False, "error": "No tables found in HTML"}

        required_headers = ["CODIGO", "DISTRITO", "DEPARTAMENTO", "MUNICIPIO"]
        target_table = None
        target_index = None

        for i, df in enumerate(tables):
            df_columns_upper = [str(col).upper().strip() for col in df.columns]
            if all(header in df_columns_upper for header in required_headers):
                target_table = df
                target_index = i
                break
            else:
                if len(df) > 0:
                    first_row_upper = [
                        str(cell).upper().strip() for cell in df.iloc[0]
                    ]
                    if all(header in first_row_upper for header in required_headers):
                        df.columns = df.iloc[0]
                        df = df.drop(df.index[0]).reset_index(drop=True)
                        target_table = df
                        target_index = i
                        break

        if target_table is None:
            return {
                "success": False,
                "error": "No table found with required headers.",
            }

        target_table = target_table.dropna(how="all")

        return {
            "success": True,
            "data": target_table,
            "table_index": target_index,
            "total_tables": len(tables),
        }

    except Exception as e:
        return {"success": False, "error": str(e)}


def sanitize_filename(text):
    filename = text.lower().replace(" ", "_")
    return re.sub(r"[^\w_.]", "", filename)


def process_html_files_directory(input_dir, output_dir):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    files = list(input_path.glob("*.xls"))

    print(f"Found {len(files)} .xls files to process")

    successful_files = []
    failed_files = []
    all_departamentos = defaultdict(list)

    for file_path in files:
        result = parse_html_excel_file(file_path)

        if result["success"]:
            df = result["data"]

            departamentos = []
            for departamento, group in df.groupby("DEPARTAMENTO"):
                filename = f"datos_{sanitize_filename(departamento)}.csv"

                output_file = output_path / filename
                group.to_csv(output_file, index=False)

                departamentos.append(
                    {"name": departamento, "filename": filename, "rows": len(group)}
                )

                all_departamentos[departamento].append(
                    {
                        "source_file": file_path.name,
                        "csv_file": filename,
                        "rows": len(group),
                    }
                )

            successful_files.append(
                {
                    "file": file_path.name,
                    "departamentos": departamentos,
                    "total_rows": len(df),
                }
            )

            print(f"  ✅ Processed {file_path.name}: Created {len(departamentos)} CSVs.")

        else:
            failed_files.append({"file": file_path.name, "error": result["error"]})
            print(f"  ❌ Failed {file_path.name}: {result['error']}")

    print("\n" + "=" * 60)
    print("PROCESSING SUMMARY")
    print("=" * 60)

    print(f"\nTotal files: {len(files)}")
    print(f"✅ Successful: {len(successful_files)}")
    print(f"❌ Failed: {len(failed_files)}")

    # The detailed successful/failed files lists will only be printed if there are successful/failed files.
    # The summary already gives a count, so the repetition for successful files is removed here.
    # The detailed list for failed files remains as it provides useful error messages.
    if failed_files:
        print(f"\n❌ FAILED FILES:")
        for item in failed_files:
            print(f"  - {item['file']}: {item['error']}")

    duplicates = {
        name: sources
        for name, sources in all_departamentos.items()
        if len(sources) > 1
    }
    if duplicates:
        print(f"\n⚠️ DUPLICATE DEPARTAMENTOS:")
        for dept_name, sources in duplicates.items():
            print(f"  {dept_name}: appears in {len(sources)} files")
    else:
        print("\n✅ No duplicate departamentos found")


test_result = parse_html_excel_file("data/raw/establecimiento.xls")
if test_result["success"]:
    print("\nSingle file test successful!")

    print("\n" + "=" * 60)
    print("PROCESSING ALL FILES")
    print("=" * 60)
    process_html_files_directory("data/raw", "data/csv")
else:
    print(f"❌ Single file test failed: {test_result['error']}")


Single file test successful!

PROCESSING ALL FILES
Found 23 .xls files to process
  ✅ Processed establecimiento (10).xls: Created 1 CSVs.
  ✅ Processed establecimiento (11).xls: Created 1 CSVs.
  ✅ Processed establecimiento (12).xls: Created 1 CSVs.
  ✅ Processed establecimiento (13).xls: Created 1 CSVs.
  ✅ Processed establecimiento (14).xls: Created 1 CSVs.
  ✅ Processed establecimiento (15).xls: Created 1 CSVs.
  ✅ Processed establecimiento (16).xls: Created 1 CSVs.
  ✅ Processed establecimiento (17).xls: Created 1 CSVs.
  ✅ Processed establecimiento (18).xls: Created 1 CSVs.
  ✅ Processed establecimiento (19).xls: Created 1 CSVs.
  ✅ Processed establecimiento (1).xls: Created 1 CSVs.
  ✅ Processed establecimiento (20).xls: Created 1 CSVs.
  ✅ Processed establecimiento (21).xls: Created 1 CSVs.
  ✅ Processed establecimiento (22).xls: Created 1 CSVs.
  ✅ Processed establecimiento (2).xls: Created 1 CSVs.
  ✅ Processed establecimiento (3).xls: Created 1 CSVs.
  ✅ Processed establecim

## .csv a DataFrames
Luego de haber creado los archivos, podemos cargarlos a DataFrames para realizar el análisis necesario.

In [42]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

# Load all CSV files
csv_dir = Path("data/csv")
csv_files = list(csv_dir.glob("*.csv"))

print(f"Found {len(csv_files)} CSV files")

# Load all datasets
datasets = {}
for csv_file in csv_files:
    dataset_name = csv_file.stem
    datasets[dataset_name] = pd.read_csv(csv_file)
    
print(f"Loaded {len(datasets)} datasets")

Found 23 CSV files
Loaded 23 datasets


# Descripción

## Filas y Columnas

In [43]:
# Check shapes of all datasets
shape_info = []
for name, df in datasets.items():
    shape_info.append({
        'dataset': name,
        'rows': df.shape[0],
        'columns': df.shape[1]
    })

shape_df = pd.DataFrame(shape_info)
print("📊 Dataset Shapes:")
print(shape_df.sort_values('rows', ascending=False))

# Summary statistics
print(f"\n📈 Summary:")
print(f"Total rows across all datasets: {shape_df['rows'].sum():,}")
print(f"Average rows per dataset: {shape_df['rows'].mean():.0f}")
print(f"Min/Max rows: {shape_df['rows'].min()} / {shape_df['rows'].max()}")

📊 Dataset Shapes:
                 dataset  rows  columns
19       datos_guatemala  1036       17
16  datos_ciudad_capital   864       17
7       datos_san_marcos   431       17
18       datos_escuintla   393       17
3   datos_quetzaltenango   365       17
14   datos_chimaltenango   300       17
1          datos_jutiapa   296       17
11   datos_suchitepequez   296       17
20   datos_huehuetenango   295       17
22    datos_alta_verapaz   294       17
21          datos_izabal   273       17
5       datos_retalhuleu   272       17
2            datos_peten   270       17
6     datos_sacatepequez   208       17
4           datos_quiche   184       17
15      datos_chiquimula   136       17
8       datos_santa_rosa   133       17
0           datos_jalapa   121       17
9           datos_solola   111       17
17     datos_el_progreso    97       17
10    datos_baja_verapaz    94       17
13          datos_zacapa    70       17
12     datos_totonicapan    51       17

📈 Summary:
Total rows

## Integridad de los Datos

### Consistencia en Nombres de Columnas

In [44]:
# Check if all datasets have the same columns
all_columns = []
column_consistency = {}

for name, df in datasets.items():
    columns = list(df.columns)
    all_columns.append(columns)
    column_consistency[name] = columns

# Check if all column sets are identical
first_columns = all_columns[0]
all_same = all(columns == first_columns for columns in all_columns)

if all_same:
    print(f"\n✅ Standard columns ({len(first_columns)}):")
    for i, col in enumerate(first_columns, 1):
        print(f"  {i:2d}. {col}")
else:
    print("\n❌ Column differences found:")
    for name, columns in column_consistency.items():
        if columns != first_columns:
            print(f"  {name}: {columns}")


✅ Standard columns (17):
   1. CODIGO
   2. DISTRITO
   3. DEPARTAMENTO
   4. MUNICIPIO
   5. ESTABLECIMIENTO
   6. DIRECCION
   7. TELEFONO
   8. SUPERVISOR
   9. DIRECTOR
  10. NIVEL
  11. SECTOR
  12. AREA
  13. STATUS
  14. MODALIDAD
  15. JORNADA
  16. PLAN
  17. DEPARTAMENTAL


### Encoding Problemático
Como mencionamos anteriormente, el encoding de los archivos originales era distinto de "utf-8". Nos dimos cuenta al realizar el análisis sobre el encoding problemático, sin embargo al cambiarlo dentro de la función anterior logramos correr con éxito este análisis.

In [45]:
import pandas as pd
from collections import defaultdict
import re

# Check for encoding issues (corrupted characters)
problematic_char = "�"
all_problematic_samples = defaultdict(list)
issue_found = False

for dataset_name, df in datasets.items():
    for col in df.columns:
        if df[col].dtype == "object":  # Only check text columns
            str_series = df[col].astype(str) 
            
            # Identify rows with the problematic character '�'
            contains_char_mask = str_series.str.contains(problematic_char, na=False)
            
            if contains_char_mask.any(): # If any '�' found in this column
                issue_found = True
                # Get unique problematic values, up to 5, for this column
                current_samples = str_series[contains_char_mask].unique().tolist()
                for sample_val in current_samples:
                    if len(all_problematic_samples[col]) < 5:
                        all_problematic_samples[col].append(sample_val)


if issue_found:
    print(f"❌ Encoding issues found (char: '{problematic_char}'):")
    # Sort columns by name for consistent output
    sorted_cols_with_issues = sorted(all_problematic_samples.keys()) 

    for col in sorted_cols_with_issues:
        samples = all_problematic_samples[col]
        print(f"  Column '{col}':")
        for val in samples:
            print(f"    • {val}")
else:
    print(f"✅ No encoding issues found (char: '{problematic_char}')")

✅ No encoding issues found (char: '�')


# Análisis de Variables
Las variables que más operaciones de limpieza necesitan son:

## Código
Este valor parece ser un identificador único, queremos explorar las siguientes propiedades:
- Unicidad: Este código es único dentro de su respectivo dataset o todos?
- Formato: Es consistente el formato en todos los datasets? Existen errores de digitación?
- Valores Faltantes: Existen valores faltantes?

Primero, podemos verificar si es necesario realizar transformaciones para evitar errores de digitación. Utilizando Regex, verificamos que siga un patrón estricto para garantizar consistencia.

In [46]:
import pandas as pd
import re
from collections import defaultdict

def verify_codigo_format_only(datasets: dict):

    strict_codigo_pattern = re.compile(r'^\d{1,2}-\d{1,3}-\d{1,4}-\d{1,3}$')

    format_issues_by_dataset = defaultdict(list)

    for name, df in datasets.items():

        # Use the raw string representation without stripping
        codigo_series_raw = df['CODIGO'].astype(str)

        # Iterate over non-null codes to check format and spaces
        for idx, code in codigo_series_raw.dropna().items():
            if not strict_codigo_pattern.match(code):
                # If it doesn't match the strict pattern at all
                format_issues_by_dataset[name].append(f"'{code}' (fila {idx}) - No coincide con el formato estricto (XX-YY-ZZZZ-AA)")
            elif code != code.strip():
                # If it matches the pattern but has leading/trailing spaces
                format_issues_by_dataset[name].append(f"'{code}' (fila {idx}) - Contiene espacios en blanco al inicio/final")

    if not format_issues_by_dataset:
        print("  ✅ Todos los códigos cumplen con el formato estricto (XX-YY-ZZZZ-AA) y no tienen espacios en blanco iniciales/finales.")
    else:
        print("  ❌ Se encontraron problemas de formato o espacios en blanco:")
        for name, issues in format_issues_by_dataset.items():
            print(f"\nDataset: '{name}' ({len(issues)} problemas)")
            for i, issue in enumerate(issues[:10]):
                print(f"  • {issue}")
            if len(issues) > 10:
                print(f"  ... y {len(issues) - 10} problemas más.")
    

verify_codigo_format_only(datasets)

  ✅ Todos los códigos cumplen con el formato estricto (XX-YY-ZZZZ-AA) y no tienen espacios en blanco iniciales/finales.


Luego, podemos verificar valores faltantes, entradas duplicadas, etc.

In [47]:
import pandas as pd
import re
from collections import defaultdict

def verify_codigo_cleanliness(datasets: dict):

    all_codigo_values_cleaned = []
    codigo_summary_by_dataset_cleaned = defaultdict(dict)
    
    codigo_pattern = re.compile(r'(\d{1,2})-(\d{1,3})-(\d{1,4})-(\d{1,3})')

    for name, df in datasets.items():

        codigo_series_cleaned = df['CODIGO'].astype(str).str.strip()

        # Drop truly missing values for uniqueness/segment analysis
        valid_codigo_cleaned = codigo_series_cleaned.dropna()

        # 1. Uniqueness within dataset (on cleaned data)
        unique_count = valid_codigo_cleaned.nunique()
        total_valid_count = len(valid_codigo_cleaned)
        missing_count_after_clean = codigo_series_cleaned.isnull().sum()
        
        duplicates_in_dataset = valid_codigo_cleaned.duplicated(keep=False).sum() # Count all occurrences of duplicates

        codigo_summary_by_dataset_cleaned[name]['total_entries_original'] = len(df)
        codigo_summary_by_dataset_cleaned[name]['missing_count'] = missing_count_after_clean
        codigo_summary_by_dataset_cleaned[name]['missing_percentage'] = (missing_count_after_clean / len(df) * 100) if len(df) > 0 else 0
        codigo_summary_by_dataset_cleaned[name]['unique_count'] = unique_count
        codigo_summary_by_dataset_cleaned[name]['duplicates_count'] = duplicates_in_dataset
        codigo_summary_by_dataset_cleaned[name]['is_unique_within_dataset'] = (duplicates_in_dataset == 0) and (missing_count_after_clean == 0)

        # Collect all cleaned codes for cross-dataset analysis
        all_codigo_values_cleaned.extend(valid_codigo_cleaned.tolist())

        # 2. Format Consistency (after stripping) and Segment Analysis
        # Any remaining format violations here are structural, not just spaces
        format_violations_cleaned = []
        segment_counts_cleaned = defaultdict(lambda: defaultdict(int))
        
        for code in valid_codigo_cleaned:
            match = codigo_pattern.match(code)
            if not match:
                format_violations_cleaned.append(code)
            else:
                parts = match.groups()
                segment_counts_cleaned['segment_1'][parts[0]] += 1
                segment_counts_cleaned['segment_2'][parts[1]] += 1
                segment_counts_cleaned['segment_3'][parts[2]] += 1
                segment_counts_cleaned['segment_4'][parts[3]] += 1

        codigo_summary_by_dataset_cleaned[name]['format_violations_cleaned'] = format_violations_cleaned
        codigo_summary_by_dataset_cleaned[name]['segment_analysis_cleaned'] = {
            segment: dict(sorted(counts.items(), key=lambda item: item[1], reverse=True)[:5])
            for segment, counts in segment_counts_cleaned.items()
        }
        
    print("\n--- Resumen de Limpieza de 'CODIGO' por Dataset ---")
    for name, summary in codigo_summary_by_dataset_cleaned.items():
        print(f"\nDataset: '{name}'")
        print(f"  • Total entradas (original): {summary['total_entries_original']}")
        print(f"  • Valores faltantes (después de limpieza): {summary['missing_count']} ({summary['missing_percentage']:.2f}%)")
        print(f"  • Valores únicos (después de limpieza): {summary['unique_count']}")
        print(f"  • Entradas duplicadas (después de limpieza): {summary['duplicates_count']}")
        print(f"  • Es único en este dataset (sin duplicados/faltantes): {'✅ Sí' if summary['is_unique_within_dataset'] else '❌ No'}")
        
        if summary['format_violations_cleaned']:
            print(f"  • Violaciones de formato (después de limpieza, {len(summary['format_violations_cleaned'])}): {summary['format_violations_cleaned'][:5]}{'...' if len(summary['format_violations_cleaned']) > 5 else ''}")
        else:
            print(f"  • Consistencia de formato (después de limpieza): ✅ Formato esperado (XX-YY-ZZZZ-AA) en todas las entradas válidas.")

        print("  • Análisis de segmentos (Top 5 en datos limpios):")
        for segment_name, top_parts in summary['segment_analysis_cleaned'].items():
            print(f"    - {segment_name}: {top_parts}")

    # 3. Cross-Dataset Uniqueness and Overlap (on cleaned data)
    print("\n--- Análisis de 'CODIGO' entre Datasets (en datos limpios) ---")
    if all_codigo_values_cleaned:
        total_codes_across_datasets = len(all_codigo_values_cleaned)
        unique_codes_across_datasets = pd.Series(all_codigo_values_cleaned).nunique()
        duplicates_across_datasets = total_codes_across_datasets - unique_codes_across_datasets

        print(f"  • Total de códigos limpios recolectados: {total_codes_across_datasets}")
        print(f"  • Códigos únicos entre todos los datasets (limpios): {unique_codes_across_datasets}")
        print(f"  • Códigos duplicados (limpios, aparecen en más de un lugar o repetidos en el mismo dataset): {duplicates_across_datasets}")
        
        if duplicates_across_datasets > 0:
            print("  • La variable 'CODIGO' NO es única a través de todos los datasets combinados (después de limpieza).")
            duplicate_series = pd.Series(all_codigo_values_cleaned)
            global_duplicates = duplicate_series[duplicate_series.duplicated(keep='first')].unique().tolist()
            if global_duplicates:
                print(f"  • Ejemplos de códigos duplicados globalmente (limpios): {global_duplicates[:10]}{'...' if len(global_duplicates) > 10 else ''}")
        else:
            print("  ✅ La variable 'CODIGO' es única a través de todos los datasets combinados (después de limpieza).")
    else:
        print("  No se encontraron códigos 'CODIGO' válidos para analizar globalmente después de la limpieza.")

    print("=" * 70)


verify_codigo_cleanliness(datasets)


--- Resumen de Limpieza de 'CODIGO' por Dataset ---

Dataset: 'datos_jalapa'
  • Total entradas (original): 121
  • Valores faltantes (después de limpieza): 0 (0.00%)
  • Valores únicos (después de limpieza): 121
  • Entradas duplicadas (después de limpieza): 0
  • Es único en este dataset (sin duplicados/faltantes): ✅ Sí
  • Consistencia de formato (después de limpieza): ✅ Formato esperado (XX-YY-ZZZZ-AA) en todas las entradas válidas.
  • Análisis de segmentos (Top 5 en datos limpios):
    - segment_1: {'21': 121}
    - segment_2: {'01': 63, '07': 17, '03': 12, '06': 11, '05': 8}
    - segment_3: {'0111': 2, '0036': 2, '0052': 2, '0054': 2, '0101': 1}
    - segment_4: {'46': 121}

Dataset: 'datos_jutiapa'
  • Total entradas (original): 296
  • Valores faltantes (después de limpieza): 0 (0.00%)
  • Valores únicos (después de limpieza): 296
  • Entradas duplicadas (después de limpieza): 0
  • Es único en este dataset (sin duplicados/faltantes): ✅ Sí
  • Consistencia de formato (despué

Respondiendo a las preguntas anteriores
- Unicidad: Todos los códigos dentro del dataset son únicos.
- Formato: El formato es consistente, sin embargo la función de por si realizó algunas operaciones de limpieza como remover los espacios en blanco.
- Valores Faltantes: No existen valores faltantes

Adicionalmente, podemos seguir explorando para determinar si el segmento 2 tiene alguna otra propiedad. Este se parece repetir a lo largo de diferentes departamentos, podría ser indicador del municipio

In [ ]:
import pandas as pd
import re
from collections import defaultdict

def validate_codigo_municipio_mapping_individual(datasets: dict):

    codigo_pattern = re.compile(r'(\d{1,2})-(\d{1,3})-(\d{1,4})-(\d{1,3})')
    all_datasets_consistent = True # Flag to track overall consistency

    for dataset_name, df in datasets.items():

        # Extract Segment 2 and clean MUNICIPIO name
        temp_df = df[['CODIGO', 'MUNICIPIO']].copy()
        temp_df['CODIGO_STRIP'] = temp_df['CODIGO'].astype(str).str.strip()
        temp_df['MUNICIPIO_CLEAN'] = temp_df['MUNICIPIO'].astype(str).str.strip().str.upper() 

        # Filter out malformed/missing CODIGO entries for mapping logic
        valid_codigo_rows = temp_df['CODIGO_STRIP'].str.match(codigo_pattern).fillna(False)
        temp_df_valid = temp_df[valid_codigo_rows].copy()

        if temp_df_valid.empty:
            print(f"  Dataset '{dataset_name}': No hay entradas de 'CODIGO' válidas para analizar la relación con 'MUNICIPIO'.")
            all_datasets_consistent = False
            continue

        temp_df_valid['CODIGO_SEGMENT2'] = temp_df_valid['CODIGO_STRIP'].str.extract(codigo_pattern)[1] 

        # Build mappings
        municipio_to_segment2 = temp_df_valid.groupby('MUNICIPIO_CLEAN')['CODIGO_SEGMENT2'].agg(lambda x: tuple(sorted(x.unique())))
        segment2_to_municipio = temp_df_valid.groupby('CODIGO_SEGMENT2')['MUNICIPIO_CLEAN'].agg(lambda x: tuple(sorted(x.unique())))

        # Identify inconsistencies
        issues_in_current_dataset = []

        # Check 1: Municipio name associated with multiple Segment 2 codes (less common for clean data)
        for municipio, segment_codes in municipio_to_segment2.items():
            if len(segment_codes) > 1:
                issues_in_current_dataset.append(
                    f"  - Municipio '{municipio}' asociado a múltiples códigos de segmento 2: {segment_codes}"
                )
        
        # Check 2: Segment 2 code associated with multiple Municipio names
        for segment_code, municipios in segment2_to_municipio.items():
            if len(municipios) > 1:
                issues_in_current_dataset.append(
                    f"  - Código de segmento 2 '{segment_code}' asociado a múltiples nombres de municipio: {municipios}"
                )
        
        # Report for the current dataset
        if issues_in_current_dataset:
            print(f"  ❌ Dataset '{dataset_name}': INCONSISTENCIAS DETECTADAS en la relación CODIGO (Segmento 2) y MUNICIPIO:")
            for issue in issues_in_current_dataset[:5]: # Limit issue examples
                print(f"    {issue}")
            if len(issues_in_current_dataset) > 5:
                print(f"    ...y {len(issues_in_current_dataset) - 5} más.")
            all_datasets_consistent = False
        else:
            # Concise success message if no issues
            print(f"  ✅ Dataset '{dataset_name}': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.")
        
    if all_datasets_consistent:
        print("Validación individual de CODIGO (Segmento 2) vs MUNICIPIO completada y todos los datasets son consistentes")
    else:
        print("Validación individual de CODIGO (Segmento 2) vs MUNICIPIO completada. Se encontraron algunas inconsistencias.")


validate_codigo_municipio_mapping_individual(datasets)

  ✅ Dataset 'datos_jalapa': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_jutiapa': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_peten': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_quetzaltenango': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_quiche': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_retalhuleu': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_sacatepequez': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_san_marcos': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_santa_rosa': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_solola': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.
  ✅ Dataset 'datos_baja_verapaz': Consistente en la relación CODIGO (Segmento 2) y MUNICIPIO.


Ahora podemos afirmar que el código se compone de [Codigo Departamento]-[Codigo Municipio]-[Identificador]-46. En conclusión, el código sigue un formato consistente y carece de errores de digitación. Luego de indagar en otras variables, se puede utilizar como una "fuente de verdad" para arreglar errores de digitación en otras columnas y asegurarnos que la información se mantenga fiel al dataset original luego de realizar más operaciones de limpieza.

## Pasos de Limpieza
Ya que el código se encuentra formateado consistentemente, la única operación por el momento sería utilizar [Codigo Departamento]-[Codigo Municipio] para construir un identificador único de municipios.

## Distrito